In [1]:
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from datetime import datetime
import math

def amazon_reviews(asin):
    global titles, stars, dates, contents, reviewer, bs_obj
    options=Options()
# 셀레니움 돌아갈 때 크롬창 안띄우는거
#     options.add_argument("--headless")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("disable-gpu")
# 다음 페이지 넘어가면 크롬 설정 한글로 바뀜, 기본 설정 영어로
    options.add_argument("--lang=en")
#     Path : Chromedriver.exe 경로
    driver=wd.Chrome(executable_path = 'E:/Amazon review data (2018)/02. sample/chromedriver.exe',options=options)

    url=f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'

    driver.get(url)
    driver.implicitly_wait(10)
    res=driver.page_source
    obj=bs(res,'html.parser')

#     총 리뷰수 확인하기
    rev=obj.select('#filter-info-section > div')[0].get_text().strip()
#     | or , 로 구분됨
    if '|' in rev:
        rev = rev.split('|')[1]
        rev = re.sub(r'[^0-9]', '', rev)
    elif ', ' in rev:
        rev = rev.split(', ')[1]
        rev = re.sub(r'[^0-9]', '', rev)

#     리뷰 제목
    titles=[]
#     별점
    stars=[]
#     리뷰 작성 날짜
    dates=[]
#     리뷰 내용
    contents=[]
#     리뷰 작성자
    reviewer = []
    
    page = 1
    
    print('전체 리뷰 ', rev, " 개")
    print('전체 ', math.ceil(int(rev)/10), '페이지')
    

    while len(titles)<int(rev):
         
#         3초간 프로세스 정지
        time.sleep(3)
        source=driver.page_source
        bs_obj=bs(source,"html.parser")
            
        
#         리뷰 타이틀 가져오기
#         미국 리뷰와 other country리뷰 태그 다름
        for i in bs_obj.find("div", {'id' :'cm_cr-review_list'}).findAll(["span", "a"],{'data-hook':'review-title'}):
            titles.append(i.get_text().strip())
            
#         ##리뷰 아이디 가져오기
        for c in bs_obj.findAll('div',{'class':'a-section review aok-relative'}):
            reviewer.append(c.find("span", class_="a-profile-name").text)
        
        
#         리뷰 날짜 가져오기
        for d in bs_obj.findAll('span',{'data-hook':'review-date'}):
            dd=''.join(d.get_text().split(' '))
            try:
                dd = dd.replace('January','01').replace('February','02').replace('March','03').replace('April','04').replace('May','05').replace('June','06').replace('July','07').replace('August','08').replace('September','09').replace('October','10').replace('November','11').replace('December','12')
            
            except Exception as e:
                print (e)
            
            dd=re.sub(r'[^0-9]', '', dd)
#             p = re.compile('[0-9]{4}년[0-9]{1,2}월[0-9]{1,2}일')
#             dates.append(p.findall(nn))
            dates.append(dd)
        
#         리뷰 내용 가져오기
        for a in bs_obj.findAll('span',{'data-hook':'review-body'}):
            contents.append(a.get_text().strip())
        
#         리뷰 평점 가져오기
        for u in bs_obj.findAll('i',{'data-hook':'review-star-rating'}):
            stars.append(u.get_text()[0])
            
        for u in bs_obj.findAll('i',{'data-hook':'cmps-review-star-rating'}):
            stars.append(int(u.get_text()[0])) 
        
        
#         코드 돌아가는 동안 페이지 확인
        if len(titles) < int(rev):
            print(page, 'page')
            page += 1
            
        if len(titles) ==rev:
            break
        
#         #셀레니움이 다음 페이지 넘어감, Next Button 클릭
        try:
            driver.find_element(by=By.CSS_SELECTOR, value='#cm_cr-pagination_bar > ul > li.a-last > a').click()
            time.sleep(1)
        except Exception as e:
            print (e)
            break
        
#     driver.close()
#     driver.quit()        

    print(len(reviewer))
    print(len(dates))
    print(len(stars))
    print(len(titles))
    print(len(contents))
#     데이터 프레임 작성
    df=pd.DataFrame({'Asin':f'{asin}','ID':reviewer,'Date':dates,'Rating':stars,"Title":titles, "Body":contents})
    
    return df

data=amazon_reviews('B08CZ5NLM1')

data.to_csv('e:/Amazon review data (2018)/100. amazon review data/B08CZ5NLM1.csv', encoding='utf-8-sig')

C:\Users\qudwh\AppData\Local\Temp/ipykernel_18156/2693362153.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=wd.Chrome(executable_path = 'E:/Amazon review data (2018)/02. sample/chromedriver.exe',options=options)


전체 리뷰  303  개
전체  31 페이지
1 page
2 page
3 page
4 page


KeyboardInterrupt: 